In [72]:
import pandas as pd
import numpy as np

from itertools import combinations

import scipy
from scipy.stats import pearsonr
from scipy.stats import spearmanr

In [73]:
pd.set_option('display.max_columns', 500)

In [74]:
data = pd.read_csv('churn_analysis.csv',  index_col = [0])
print(data.shape)
data.head()

(3333, 22)


,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [75]:
data['churn'].replace({'False.' : 0, 'True.' : 1}, inplace = True)

In [76]:
control_group = data[data['treatment'] == 1]

Проверим гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора

Для начала создадим одну таблицу сопряженности для номинального признака 'state' и целевой переменной 'churn'

In [77]:
contingency_table = pd.pivot_table(control_group[['state', 'churn']], index=['state'], columns=['churn'], aggfunc =len, fill_value = 0)
contingency_table

churn,0,1
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5
CO,17,5
CT,25,5
DC,18,1
DE,16,1


Теперь построим таблицы сопряженности для каждой пары штатов и значением признака churn

In [78]:
contingency_table_array = np.c_[contingency_table[0].values, contingency_table[1].values]

In [79]:
pairs = list(combinations(contingency_table_array, 2))

In [80]:
pairs = np.array(pairs)

In [81]:
pairs[:3]

array([[[19,  1],
        [25,  5]],

       [[19,  1],
        [11,  5]],

       [[19,  1],
        [17,  2]]], dtype=int64)

Проверим гипотезу независимости признаков с помощью критерия ꭓ2, который позволяет проверить наличие связи между двумя признаками на основании эмпирических частот

In [82]:
chi2_pvalue_nocorr = []
for i in range(pairs.shape[0]):
    chi2_pvalue_nocorr.append(scipy.stats.chi2_contingency(pairs[i], correction=False)[1])

In [83]:
chi_nc = [x for x in chi2_pvalue_nocorr if x < 0.05]
len(chi_nc)

34

Видим, что только в 34 случаях из 1275 p-value стало меньше 0.05, что говорит о наличии зависимости признаков. В большинстве же случаев достигаемые уровни значимости больше 0.05, а следовательно, у нас нет оснований отклонить нулевую гипотезу о независимости признаков штата и ухода клиента

Посчитаем корреляции Пирсона и Спирмена между признаками 'day_calls' и 'mes_estim'

In [84]:
pearsonr(data['day_calls'], data['mes_estim'])

(-0.051794350587572625, 0.0027798836869756707)

In [85]:
spearmanr(data['day_calls'], data['mes_estim'])

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

Оба метода возвращают вторым элементом значение p-value, доверие к которому растет с увеличением объема выборки более 500 ( в нашем случае 3333). В обоих случаях достигаемые уровни значимости оказались меньше 0.05, что говорит о том, что мы отвергаем нулевую гипотезу об отсутствии корреляций между признаками

Посчитаем значение p-value для критерия ꭓ2 о независимости признаков 'state' и 'churn'.  Видим, что оно получилось больше 0.05, следовательно, у нас нет оснований отклонить нулевую гипотезу о независимости признаков

In [86]:
scipy.stats.chi2_contingency(contingency_table)[1]

0.70975900427784411

Теперь найдем значение коэффицента корреляции Крамера между штатом и оттоком пользователей

In [87]:
corrV = np.sqrt((scipy.stats.chi2_contingency(contingency_table)[0])/((contingency_table.sum().sum())*(np.min(contingency_table.shape) -1)))

In [88]:
corrV

0.20039321502033319

С помощью проверки гипотезы о равенстве долей проведем анализ эффективности удержания клиентов различными методиками. (В выводе p-value учтено, что альтернативная гипотеза считается 'smaller')

In [89]:
from statsmodels.stats.proportion import proportions_ztest
count = np.array([data[data['treatment'] == 0]['churn'].value_counts()[0], data[data['treatment'] == 1]['churn'].value_counts()[0]])
nobs = np.array([data[data['treatment'] == 0]['churn'].value_counts().sum(), data[data['treatment'] == 1]['churn'].value_counts().sum()])
z,p = proportions_ztest(count, nobs, value=0, alternative= 'smaller')
print('p-value = {p}'.format(p= 1-p))

p-value = 0.11416558195225623


In [90]:
from statsmodels.stats.proportion import proportions_ztest
count = np.array([data[data['treatment'] == 2]['churn'].value_counts()[0], data[data['treatment'] == 1]['churn'].value_counts()[0]])
nobs = np.array([data[data['treatment'] == 2]['churn'].value_counts().sum(), data[data['treatment'] == 1]['churn'].value_counts().sum()])
z,p = proportions_ztest(count, nobs, value=0, alternative= 'smaller')
print('p-value = {p}'.format(p=1-p))

p-value = 0.0046740421472255544


Видим, что для treatment = 0 альтернативная гипотеза отклоняется, значит, доли ушедших клиентов равны, что говорит о безуспешности методики. Однако для treatment = 2 p-value получилось менее 0.05, что говорит нам о том, что мы находимся в критической области, а значит, доля ушедших клиентов при втором методе меньше. Вероятно, данный метод можно порекомендовать компании, поскольку в одном случае он позволяет избежать более высокого процента оттока клиентов.

Сравним эти два метода между собой на равенство долей (альтернативная будет 'two-sided')

In [91]:
count = np.array([data[data['treatment'] == 2]['churn'].value_counts()[0], data[data['treatment'] == 0]['churn'].value_counts()[0]])
nobs = np.array([data[data['treatment'] == 2]['churn'].value_counts().sum(), data[data['treatment'] == 0]['churn'].value_counts().sum()])
z,p = proportions_ztest(count, nobs, value=0, alternative='two-sided')
print('p-value = {p}'.format(p=p))

p-value = 0.1564246886050785


Видим, что нет оснований отклонить нулевую гипотезу о равенстве долей, поскольку достигаемый уровень значимости оказался выше 0.05